# Most Frequently Used next-app prediction

In [9]:
from torch import optim
from data.dataloader import DatasetLoader, PretrainLoader
from torch.utils.data import DataLoader
import torch
from models.BiGRU import BiGRU
from models.PreTrainEmbedding import PreTrainEmbedding
from torch import nn
from tqdm import tqdm
import pandas as pd
import numpy as np

## 1. Dataset uitlezen

In [10]:
removeapps = ["Screen on (locked)",
              "Screen off (locked)",
              "Screen on (unlocked)",
              "Screen off",
              "Samsung Experience Service",
              "Package installer",
              "System UI",
              "Customisation Service",
              "Configuration update",
              "EmergencyManagerService",
              "DeviceKeystring",
              "Samsung Keyboard",
              "HwModuleTest",
              "Device shutdown",
              "Device boot"]
dataset = DatasetLoader("eventlog/phone_usage_cleaned.csv",
                        seq_length=1)
dataset.clean(removeapps=removeapps)
train_dataloader = DataLoader(dataset)

## initialiseer MRU waarden na 2000 eerste applicaties

In [33]:
mru = np.zeros(shape=(5,), dtype=np.int) # value==app_index
nr_opened_apps=0

for prev_app_index, _ in train_dataloader:

    if prev_app_index.item() in mru:
        mru = mru[mru!=prev_app_index.item()]
        mru = np.append(mru, prev_app_index, axis=0)
    else:
        mru[0:4] = mru[1:5]
        mru[4] = prev_app_index.item()

    nr_opened_apps+=1
    if nr_opened_apps==2000:
        print(mru)
        break

[18 21 11  8 24]


## Maak predicties en update MFU waarden voor de rest van de applicaties

In [36]:
nr_opened_apps = 0
running_accuracy_list = []

for _, next_app_index in train_dataloader:

    nr_opened_apps+=1
    if nr_opened_apps > 2000:
        running_accuracy_list.append(next_app_index.item() in mru)

        if next_app_index.item() in mru:
            mru = mru[mru!=next_app_index.item()]
            mru = np.append(mru, next_app_index, axis=0)
        else:
            mru[0:4] = mru[1:5]
            mru[4] = next_app_index.item()

print(running_accuracy_list)

[True, False, True, True, True, False, False, False, False, False, True, False, False, True, True, True, True, False, False, False, True, False, False, False, True, False, False, False, True, False, True, True, True, True, True, False, True, True, True, True, True, True, False, False, True, True, False, True, True, True, True, False, False, True, True, True, True, True, True, True, True, False, True, True, False, True, True, True, False, True, True, True, True, True, True, False, False, False, True, True, False, False, False, False, True, True, False, False, True, True, False, True, False, False, True, True, False, False, False, True, True, False, True, False, False, False, True, True, True, True, True, True, False, True, True, True, True, False, False, True, True, True, True, True, False, True, True, True, True, True, True, True, False, False, False, False, True, True, True, True, True, True, False, False, False, True, True, True, False, False, False, True, True, True, True, True, Tru

## Accuracy uitschrijven naar comparison

In [39]:
AccuracyComparison = pd.read_csv("outputdata/accuracycomparison.csv", index_col=False)
AccuracyComparison["MRU"] = running_accuracy_list
AccuracyComparison.to_csv("outputdata/accuracycomparison.csv", index=False)
AccuracyComparison.head()


,MFU,MRU
0,True,True
1,False,False
2,False,True
3,False,True
4,False,True
